In [10]:
import pandas as pd
import grblogtools as glt
from report_functions import *

In [25]:
samp_100_dyn_df = pd.read_csv("/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_100samples/dynamic_linear/ld_medium_instance_new_config_S4_results.csv")
samp_50_dyn_df = pd.read_csv("/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_50samples/dynamic_linear/ld_medium_instance_new_config_S4_results.csv")
samp_50_dyn_df.head(100)


# def add_log_and_gap(df, log_file_location, file_name_stuff):
#     df = read_csv_add_log( log_file_location, file_name_stuff, df)
#     df = read_csv_add_gap(df, gapname="gap_dyn")
#     return df

# samp_100_dyn_df = add_log_and_gap(samp_100_dyn_df, 
#                                   "/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_50samples/dynamic_linear/", 
#                                   "ld_medium_instance_new_config_S40")
log_file_location_50 = "/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_50samples/dynamic_linear/"
file_name_stuff_50 = "ld_medium_instance_new_config_S4"
log_file_location_100 = "/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_100samples/dynamic_linear/"
file_name_stuff_100 = "ld_medium_instance_new_config_S4"
def add_log_and_gap(log_file_location, file_name_stuff, df):
    df = read_csv_add_log( log_file_location, file_name_stuff, df)
    df = read_csv_add_gap(df, gapname="gap_dyn")
    return df
samp_100_dyn_df = add_log_and_gap(log_file_location_100, file_name_stuff_100, samp_100_dyn_df)
samp_50_dyn_df = add_log_and_gap(log_file_location_50, file_name_stuff_50, samp_50_dyn_df)
samp_100_dyn_df.head(100)


,Unnamed: 0,model_name,instance_name,obj_value,solver_status,equipment_instance,xp_config_file,model_file,variables_folder,run_time,group_counter,log,gap_dyn
0,0,dynamic linear labour recourse,n=50_131_n=50_132,13405.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4247.475657,0,/Users/letshopethisworks2/Documents/Phd Paper ...,46.7363
1,1,dynamic linear labour recourse,n=50_12_n=50_13,7487.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4307.290886,1,/Users/letshopethisworks2/Documents/Phd Paper ...,45.4788
2,2,dynamic linear labour recourse,n=50_14_n=50_15,13127.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4247.077502,2,/Users/letshopethisworks2/Documents/Phd Paper ...,64.4016
3,3,dynamic linear labour recourse,n=50_19_n=50_20,6828.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4321.156084,3,/Users/letshopethisworks2/Documents/Phd Paper ...,26.0838
4,4,dynamic linear labour recourse,n=50_1_n=50_2_n=50_3,8688.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4243.335348,4,/Users/letshopethisworks2/Documents/Phd Paper ...,49.0331
5,5,dynamic linear labour recourse,n=50_9_n=50_10_n=50_11,11685.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4243.043825,5,/Users/letshopethisworks2/Documents/Phd Paper ...,61.7629
6,6,dynamic linear labour recourse,n=50_21_n=50_22_n=50_23,9715.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4241.828736,6,/Users/letshopethisworks2/Documents/Phd Paper ...,56.1194
7,7,dynamic linear labour recourse,n=50_53_n=50_54_n=50_55,14049.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4242.116869,7,/Users/letshopethisworks2/Documents/Phd Paper ...,50.8719
8,8,dynamic linear labour recourse,n=50_54_n=50_55_n=50_56,14123.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4243.991755,8,/Users/letshopethisworks2/Documents/Phd Paper ...,54.6272


## reading single run result logs